# Collaborative Filtering Recommender System - Expedia Hotel dataset

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
import warnings
import implicit
from scipy.sparse import coo_matrix
import scipy.sparse as sparse

import sys

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
import implicit

/Users/yas/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [59]:
hotel_train_set = pd.read_csv('../data/hotel_data/train.csv', sep=',', nrows=400000)
hotel_train_set.shape

(400000, 24)

### Read train and test data

In [60]:
hotel_train_set.head(n=2)

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,1,1,2,50,628,1


In [61]:
len(hotel_train_set['srch_destination_type_id'].unique()),len(hotel_train_set['hotel_cluster'].unique())

(8, 100)

In [62]:
df = hotel_train_set[['user_id','hotel_cluster','is_booking']]

In [63]:
df.shape

(400000, 3)

In [64]:
#rename columns
df.columns =['user_id', 'item_id', 'rating']

In [65]:
# for user 12 and item 12 we have 3 values
df.head()

,user_id,item_id,rating
0,12,1,0
1,12,1,1
2,12,1,0
3,93,80,0
4,93,21,0


### Remove rows with the same user_id and item_id but different rating

In [66]:
max_rating = df.groupby(['user_id', 'item_id']).rating.transform(max)
df = df.loc[df.rating == max_rating]
df.drop_duplicates(keep='first',inplace=True) 
df= df.reset_index().drop('index',axis=1)

In [67]:
df.shape

(160372, 3)

In [68]:
len(df['user_id'].unique())

13272

# Find Similar Hotel clusters

In [69]:
ratings = pd.DataFrame(df.groupby('item_id')['rating'].mean())
ratings.head()

,rating
item_id,
0,0.109148
1,0.259607
2,0.213814
3,0.085627
4,0.183902


In [70]:
ratings['number_ratings'] = pd.DataFrame(df.groupby('item_id')['rating'].count())
ratings.head()

,rating,number_ratings
item_id,,
0,0.109148,1585
1,0.259607,1171
2,0.213814,1969
3,0.085627,1308
4,0.183902,1789


In [71]:
hotel_matrix = df.pivot_table(index='user_id',columns='item_id',values='rating')

In [72]:
hotel_matrix.head()

item_id,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
user_id,,,,,,,,,,,,,,,,,,,,,
12,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0
192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,1.0,0.0


In [73]:
def find_similar_clusters(cluster_number,top_n):
    #Select user ratings for twohotel_matrixmovies 
    item_user_ratings = hotel_matrix[cluster_number]

    # Find correlations between series with corrwith (instead of corr)
    similar_to_hotel = hotel_matrix.corrwith(item_user_ratings)

    # Removing NaN values and using a DataFrame instead of a series 
    corr_hotel = pd.DataFrame(similar_to_hotel,columns=['Correlation'])
    corr_hotel.dropna(inplace=True)

    corr_hotel = corr_hotel.join(ratings['number_ratings'])

    result = corr_hotel[corr_hotel['number_ratings']>0].sort_values('Correlation',ascending=False).head(n=top_n)
    return result

In [92]:
warnings.filterwarnings("ignore")
find_similar_clusters(9,15)

,Correlation,number_ratings
item_id,,
9,1.000000,2226
27,0.280561,273
76,0.194046,1653
30,0.189264,2169
23,0.185466,1244
8,0.181561,1262
7,0.180252,1297
25,0.178203,2330
89,0.174317,1394


In [75]:
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from sklearn.preprocessing import MinMaxScaler

import implicit

In [89]:
#csr_matrix((data, (row, col))
sparse_item_user = sparse.csr_matrix((df['rating'].astype(float),(df['item_id'], df['user_id'])))
sparse_user_item = sparse.csr_matrix((df['rating'].astype(float),(df['user_id'], df['item_id'])))


model = implicit.als.AlternatingLeastSquares(factors=20,regularization=0.1,iterations=20)
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')
model.fit(data_conf)
item_id = 9
n_similar = 15
similar = model.similar_items(item_id,n_similar)
for item in similar:
    idx,score = item
    print(df.item_id.loc[df.item_id == idx].iloc[0])


9
96
41
25
36
11
84
89
34
19
21
5
10
28
0


user_id = 936
recommended = model.recommend(user_id,sparse_user_item)
movies = []
scores = []

for item in recommended:
    idx, score = item
    movies.append(dataset.movie_title.loc[dataset.movie_id == idx].iloc[0])
    scores.append(score)
    
recommendations = pd.DataFrame({'movies': movies, 'scores':scores})
print(recommendations)

In [85]:
user_items = sparse_user_item
recommendations = model.recommend(12, user_items)
#recommendations
# find related items
related = model.similar_items(11)


In [86]:
related

[(11, 0.52067053),
 (75, 0.26548603),
 (93, 0.24800457),
 (2, 0.23009013),
 (38, 0.22017722),
 (90, 0.21013606),
 (13, 0.20751709),
 (21, 0.20671758),
 (99, 0.17370881),
 (76, 0.16020542)]

<1198785x100 sparse matrix of type '<class 'numpy.float64'>'
	with 160372 stored elements in Compressed Sparse Row format>

# Recommendation Engine - collaborative filtering model from scratch

## Memory-Based CF by computing cosine similarity

In [32]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
n_users,n_items

(5142, 100)

In [33]:
df.head()

,user_id,item_id,rating
0,12,1,1
1,93,80,0
2,93,21,0
3,93,92,0
4,501,41,0


In [34]:
list_user = df['user_id']
list_item = df['item_id']

In [35]:
# sort dataframe based on user_id and item_id
df = df.sort_values(['user_id','item_id'])
df = df.reset_index().drop('index',axis=1)

In [36]:
df.head()

,user_id,item_id,rating
0,12,1,1
1,93,21,0
2,93,80,0
3,93,92,0
4,501,10,0


In [37]:
# apply a function to generate new indexes for user_id and item_id

def id_to_index(df):
    """
    maps the values to the lowest consecutive values
    :param df: pandas Dataframe with columns user, item, rating
    :return: pandas Dataframe with the extra columns index_item and index_user
    """

    index_item = np.arange(0, len(df.item_id.unique()))
    index_user = np.arange(0, len(df.user_id.unique()))

    df_item_index = pd.DataFrame(df.item_id.unique(), columns=["item"])
    df_item_index["new_index"] = index_item
    df_user_index = pd.DataFrame(df.user_id.unique(), columns=["user"])
    df_user_index["new_index"] = index_user

    df["index_item"] = df["item_id"].map(df_item_index.set_index('item')["new_index"]).fillna(0)
    df["index_user"] = df["user_id"].map(df_user_index.set_index('user')["new_index"]).fillna(0)


    return df

In [38]:
df = id_to_index(df)

In [39]:
# split data to train and test
train, test = train_test_split(df, test_size=0.3)

train.shape, test.shape

((42665, 5), (18286, 5))

In [40]:
# create a user-item matrix which can be used to calculate the similarity between users and items
data_matrix = np.zeros((n_users, n_items))
for line in df.itertuples():
    data_matrix[line[5], line[4]] = line[3]
    
train_data_matrix = np.zeros((n_users, n_items))
# unpack the Pandas object
for line in train.itertuples():
    # adjust to count rows and cols from 0 and fill in the matrix
    train_data_matrix[line[5], line[4]] = line[3]

test_data_matrix = np.zeros((n_users, n_items))
for line in test.itertuples():
    test_data_matrix[line[5], line[4]] = line[3]

In [41]:
train_data_matrix.shape, test_data_matrix.shape

((5142, 100), (5142, 100))

In [42]:
# calculating the similarity by using the pairwise_distance from sklearn to calculate the cosine similarity
from sklearn.metrics.pairwise import pairwise_distances 

# user-user similarity
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')

# item-item similarity
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

user_similarity.shape , item_similarity.shape

((5142, 5142), (100, 100))

we can make a prediction by applying the following formula for user-based CF.


1) We can look at the similarity between users k and a as weights

2)  weights are multiplied by the ratings of a similar user a (corrected for the average rating of that user)

3) We need to normalize it so that the ratings stay between 1 and 5

4) As a final step, sum the average ratings for the user that you are trying to predict.

$$\hat{x}_{k,m}= \bar{x}_{k} + \frac{\sum_{u_{a}}sim_{u}(u_{k},u_{a})(x_{a,m}-\bar{x}_{u_{a}})}
{\sum_{u_{a}\left | sim_{u}(u_{k},u_{a}) \right |}}
$$

Also, we can make a prediction by applying the following formula for item-based CF.

 $$\hat{x}_{k,m}=  \frac{\sum_{i_{b}}sim_{i}(i_{m},i_{b})(x_{k,b})}
{\sum_{i_{b}\left | sim_{i}(i_{m},i_{b}) \right |}}
$$

In [43]:
#make predictions based on these similarities

def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #We use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [44]:
user_prediction = predict(train_data_matrix, user_similarity, type='user')
item_prediction = predict(train_data_matrix, item_similarity, type='item')

user_prediction.shape,item_prediction.shape,test_data_matrix.shape

((5142, 100), (5142, 100), (5142, 100))

## Model-Based CF by using singular value decomposition (SVD)
$$X = USV^{T}$$

In [45]:
sparsity = round(1.0-len(df)/float(n_users*n_items), 3)
print('The sparsity level of MovieLens100K is ' + str(sparsity*100) + '%')


import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_matrix, k=20)
s_diag_matrix=np.diag(s)

#prediction
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)

The sparsity level of MovieLens100K is 88.1%


## Model-Based CF by using ALS
Alternating Least Squares is a form of matrix factorization that reduces this user-item matrix to a much smaller amount of dimension called latent or hidden features

In [47]:
import implicit
#Creating Sparse Matrix
sparse_item_user = sparse.csr_matrix(train_data_matrix)
sparse_user_item = sparse.csr_matrix(train_data_matrix.T)

#Initialising The ALS model
model = implicit.als.AlternatingLeastSquares(factors=20,regularization=0.1,iterations=20)

#Calculating the Confidence by multiplying it by our Alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')
model.fit(data_conf)

user_vecs = model.user_factors
item_vecs = model.item_factors

In [48]:
item_id = 11
n_similar = 10
similar = model.similar_items(item_id,n_similar)
for item in similar:
    idx,score = item
    print(df.item_id.loc[df.item_id == idx].iloc[0])

IndexError: single positional indexer is out-of-bounds

We start out by calculating the confidence for all users and items, create our X and Y matrices to hold our user and item vectors and randomly assign the values. We also precompute our I diagonals.

In [33]:
#needs to be applied
def implicit_als(sparse_data, alpha_val=40, iterations=10, lambda_val=0.1, features=10):
 
    '''We teratively
    compute the user (x_u) and item (y_i) vectors using the following formulas:
 
    x_u = ((Y.T*Y + Y.T*(Cu - I) * Y) + lambda*I)^-1 * (X.T * Cu * p(u))
    y_i = ((X.T*X + X.T*(Ci - I) * X) + lambda*I)^-1 * (Y.T * Ci * p(i))
 '''
    '''Args:
        sparse_data (csr_matrix): Our sparse user-by-item matrix
 
        alpha_val (int): The rate in which we'll increase our confidence
        in a preference with more interactions.
 
        iterations (int): How many times we alternate between fixing and 
        updating our user and item vectors
 
        lambda_val (float): Regularization value
 
        features (int): How many latent features we want to compute.
    
    Returns:     
        X (csr_matrix): user vectors of size users-by-features
        
        Y (csr_matrix): item vectors of size items-by-features'''
   

    confidence = train_data_matrix * alpha_val

    # Get the size of user rows and item columns
    user_size, item_size = sparse_data.shape

    # We create the user vectors X of size users-by-features, the item vectors
    # Y of size items-by-features and randomly assign the values.
    X = sparse.csr_matrix(np.random.normal(size = (user_size, features)))
    Y = sparse.csr_matrix(np.random.normal(size = (item_size, features)))

    #Precompute I and lambda * I
    X_I = sparse.eye(user_size)
    Y_I = sparse.eye(item_size)

    I = sparse.eye(features)
    lI = lambda_val * I

## Evaluation with RMSE and MAE

In [34]:
user_prediction

array([[ 0.0022666 ,  0.01970385,  0.00124836, ..., -0.00239853,
         0.00254038, -0.00140417],
       [ 0.00239862,  0.00958873, -0.00881795, ..., -0.0125568 ,
        -0.00766753, -0.01140638],
       [ 0.00239862,  0.00958873, -0.00881795, ..., -0.0125568 ,
        -0.00766753, -0.01140638],
       ...,
       [ 0.02267404,  0.02966197,  0.0113555 , ...,  0.00765549,
         0.01253747,  0.00874665],
       [ 0.01260131,  0.0193384 ,  0.00147962, ..., -0.00237946,
         0.00248242, -0.00114164],
       [ 0.01243589,  0.01909461,  0.00138101, ..., -0.00224485,
         0.00271702, -0.00107735]])

In [35]:
#test_data_matrix = test_data_matrix.as_matrix()
#np.nonzero(test_data_matrix)
#test_data_matrix.nonzero()

In [36]:
test_data_matrix.shape,user_prediction.shape

((3478, 100), (3478, 100))

In [37]:
def rmse(prediction, y):
    prediction = prediction[y.nonzero()].flatten()
    y = y[y.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, y))

def mae(prediction, y):
    prediction = prediction[y.nonzero()].flatten()
    y = y[y.nonzero()].flatten()
    return sqrt(mean_absolute_error(prediction, y))

print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))
print('Model-Based CF RMSE: ' + str(rmse(X_pred, test_data_matrix)))
print("\n")
print('User-based CF MAE: ' + str(mae(user_prediction, test_data_matrix)))
print('Item-based CF MAE: ' + str(mae(item_prediction, test_data_matrix)))
print('Model-Based CF MAE: ' + str(mae(X_pred, test_data_matrix)))

User-based CF RMSE: 0.961098955066529
Item-based CF RMSE: 0.9657967908189006
Model-Based CF RMSE: 0.9720326131876091


User-based CF MAE: 0.9798780210303135
Item-based CF MAE: 0.9822933237857697
Model-Based CF MAE: 0.9838398784992692


## Evaluation with Precision and recall
Precision and recall are binary metrics used to evaluate models with binary output. 

We need a way to translate the ratings from 1 to 5 into a binary problem.

To do the translation we will assume that any true rating above 3.5 corresponds to a relevant item and any true rating below 3.5 is irrelevant. 

We are intrested in recommending top-N items to the user. So it makes more sense to compute precision and recall metrics in the first N items instead of all the items.

Thus the notion of precision and recall at k where k is a user definable integer that is set by the user to match the top-N recommendations objective.

# Resources

https://blog.cambridgespark.com/nowadays-recommender-systems-are-used-to-personalize-your-experience-on-the-web-telling-you-what-120f39b89c3c

https://course.fast.ai/videos/?lesson=4

https://towardsdatascience.com/collaborative-filtering-with-fastai-3dbdd4ef4f00

https://medium.com/quantyca/deep-learning-for-collaborative-filtering-using-fastai-b28e197ccd59

https://stackoverflow.com/questions/51171974/user-item-rating-matrix-indexerror